Imports

In [1]:
import kagglehub

In [3]:
# Craigslist
path_craigslist = kagglehub.dataset_download("michaelbryantds/bay-area-craigslist-rentals")

# Germany
path_germany = kagglehub.dataset_download("corrieaar/apartment-rental-offers-in-germany")

# Poland
path_poland = kagglehub.dataset_download("krzysztofjamroz/apartment-prices-in-poland")